In [38]:
import pandas as pd
import numpy as np
import networkx as nx
import random
from itertools import combinations
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go
import webbrowser


In [39]:

df = pd.read_csv("tum_sentetik2.csv")
df['Kategori'] = df['Mağaza'].apply(lambda x: x.split('_')[0])


available_products = df['Ürün'].unique()
product_to_category = df.groupby('Ürün')['Kategori'].first().to_dict()


def get_shopping_list(available_products, product_to_category):
    print("\n=== Alışveriş Listesi Oluşturma ===")
    shopping_list = []
    print("\nAlışveriş listesine eklemek istediğiniz ürünleri girin.")
    print("Bitirmek için 'bitir' yazın.")
    
    while True:
        product = input("Ürün adı: ").strip()
        if product.lower() == 'bitir':
            break
        if product in available_products:
            shopping_list.append(product)
            print(f"{product} listeye eklendi.")
        else:
            print(f"Hata: '{product}' veri setinde bulunmuyor. Lütfen geçerli bir ürün girin.")
    
    if not shopping_list:
        raise ValueError("Alışveriş listesi boş olamaz!")
    
    print("\nOluşturulan Alışveriş Listesi:", shopping_list)
    return shopping_list


shopping_list = get_shopping_list(available_products, product_to_category)

# Ürün kategorilerini oluştur
def create_dynamic_item_categories(shopping_list, product_to_category):
    item_categories = {}
    for item in shopping_list:
        category = product_to_category.get(item, "Bilinmeyen")
        item_categories[item] = category
    return item_categories

item_categories = create_dynamic_item_categories(shopping_list, product_to_category)

# Mağaza kategorileri için renkler 
kategori_renkleri = {
    "Gida": "orange", "Kozmetik": "purple", "Giysi": "blue", "Elektronik": "red", "Ev": "green"
}

# Ürünleri içeren mağazaları bul 
filtered_stores = df[df['Ürün'].isin(shopping_list)][['Mağaza', 'X_Koordinat', 'Y_Koordinat', 'Kategori']].drop_duplicates()
valid_stores = set(filtered_stores['Mağaza'])
store_products = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].groupby('Mağaza')['Ürün'].apply(set).to_dict()
product_prices = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].set_index(['Mağaza', 'Ürün'])['Fiyat'].to_dict()

# Ürün-Mağaza eşleştirmesi
item_to_stores = {item: set() for item in shopping_list}
for store in filtered_stores['Mağaza']:
    store_category = store.split('_')[0]
    for item in store_products.get(store, set()):
        if item in shopping_list and item_categories[item] == store_category:
            item_to_stores[item].add(store)




=== Alışveriş Listesi Oluşturma ===

Alışveriş listesine eklemek istediğiniz ürünleri girin.
Bitirmek için 'bitir' yazın.


Bant listeye eklendi.
Sabun listeye eklendi.
Süt listeye eklendi.
Peynir listeye eklendi.
Hırka listeye eklendi.
Elbise listeye eklendi.
Bot listeye eklendi.
Tablet listeye eklendi.
Oje listeye eklendi.

Oluşturulan Alışveriş Listesi: ['Bant', 'Sabun', 'Süt', 'Peynir', 'Hırka', 'Elbise', 'Bot', 'Tablet', 'Oje']


In [40]:
# Graf oluştur
G_ga = nx.Graph()
for _, row in filtered_stores.iterrows():
    G_ga.add_node(row['Mağaza'], pos_ga=(row['X_Koordinat'], row['Y_Koordinat']), kategori_ga=row['Kategori'])

for (m1, x1, y1), (m2, x2, y2) in combinations(filtered_stores[['Mağaza', 'X_Koordinat', 'Y_Koordinat']].values, 2):
    distance = euclidean((x1, y1), (x2, y2))
    travel_time = distance / 60  # Saat cinsinden
    G_ga.add_edge(m1, m2, weight=travel_time)

store_list = list(G_ga.nodes())
pos_ga = nx.get_node_attributes(G_ga, 'pos_ga')

### Genetik Algoritma Fonksiyonları

In [41]:
def initialize_population(shopping_list, item_to_stores, population_size):
    population = []
    for _ in range(population_size):
        # Ürün-mağaza eşleştirmesi
        item_to_store = {}
        selected_stores = set()
        uncovered_items = set(shopping_list)
        

        while uncovered_items:
            best_store = None
            best_coverage = 0
            for store in store_list:
                if store not in selected_stores:
                    coverage = len(store_products.get(store, set()) & uncovered_items)
                    if coverage > best_coverage:
                        best_coverage = coverage
                        best_store = store
            if best_store:
                selected_stores.add(best_store)
                for item in store_products.get(best_store, set()) & uncovered_items:
                    item_to_store[item] = best_store
                uncovered_items -= store_products.get(best_store, set())
            else:
                item = random.choice(list(uncovered_items))
                store = random.choice(list(item_to_stores[item]))
                item_to_store[item] = store
                selected_stores.add(store)
                uncovered_items.remove(item)
        
        for item in shopping_list:
            if item not in item_to_store:
                item_to_store[item] = random.choice(list(item_to_stores[item]))
        
        # Mağazaların ziyaret sırası 
        store_order = list(selected_stores)
        random.shuffle(store_order)
        
        # Birey: 
        individual = (item_to_store, store_order)
        population.append(individual)
    return population

def calculate_cost(individual, graph, product_prices, shopping_list):
    item_to_store, store_order = individual
    total_cost = 0
    travel_cost_total = 0
    item_sources = {}
    
    # Ürün fiyatlarını hesapla
    for item, store in item_to_store.items():
        price = product_prices.get((store, item), float('inf'))
        if price < float('inf'):
            item_sources[item] = (store, price)
            total_cost += price
        else:
            return float('inf'), item_sources, store_order
    
    # Mağazaları verilen sırayla ziyaret et ve ulaşım maliyetini hesapla
    if len(store_order) > 1:
        for i in range(len(store_order) - 1):
            store1, store2 = store_order[i], store_order[i + 1]
            if graph.has_edge(store1, store2):
                travel_cost = graph[store1][store2]['weight'] * 100  # Saat başına 100 TL
                travel_cost_total += travel_cost
            else:
                return float('inf'), item_sources, store_order
        total_cost += travel_cost_total
    

    if set(item_sources.keys()) != set(shopping_list):
        return float('inf'), item_sources, store_order
    
    return total_cost, item_sources, store_order

def crossover(parent1, parent2):
    item_to_store1, store_order1 = parent1
    item_to_store2, store_order2 = parent2
    
    # Ürün-mağaza eşleştirmesi için çaprazlama
    child_item_to_store = {}
    for item in item_to_store1:
        child_item_to_store[item] = random.choice([item_to_store1[item], item_to_store2[item]])
    
    # Mağaza sırası için çaprazlama (order crossover - OX)
    child_store_order = []
    selected_stores = set(child_item_to_store.values())
    
    if len(selected_stores) > 1:
        # Ebeveyn sıralarını sadece seçilen mağazalara göre filtrele
        parent1_order = [store for store in store_order1 if store in selected_stores]
        parent2_order = [store for store in store_order2 if store in selected_stores]
        
        if len(parent1_order) > 1 and len(parent2_order) > 1:
            # Rastgele iki kesim noktası seç
            length = min(len(parent1_order), len(parent2_order))
            start, end = sorted(random.sample(range(length), 2))
            
            # Parent1’den kesim parçasını al
            child_store_order = parent1_order[start:end]
            remaining = [store for store in parent2_order if store not in child_store_order]
            
            # Kalan mağazaları parent2 sırasına göre ekle
            child_store_order = remaining[:start] + child_store_order + remaining[start:]
        else:
            child_store_order = list(selected_stores)
            random.shuffle(child_store_order)
    else:
        child_store_order = list(selected_stores)
    
    return (child_item_to_store, child_store_order)

def mutation(individual, item_to_stores, mutation_rate=0.1):
    item_to_store, store_order = individual
    item_to_store = item_to_store.copy()
    store_order = store_order.copy()
    
    # Ürün-mağaza eşleştirmesi için mutasyon
    for item in item_to_store:
        if random.random() < mutation_rate:
            item_to_store[item] = random.choice(list(item_to_stores[item]))
    
    # Mağaza sırası için mutasyon (iki mağazanın yerini değiştir)
    if len(store_order) > 1 and random.random() < mutation_rate:
        i, j = random.sample(range(len(store_order)), 2)
        store_order[i], store_order[j] = store_order[j], store_order[i]
    
    # Yeni seçilen mağazalara göre sırayı güncelle
    selected_stores = set(item_to_store.values())
    store_order = [store for store in store_order if store in selected_stores]
    for store in selected_stores:
        if store not in store_order:
            store_order.append(store)
    
    return (item_to_store, store_order)

def genetic_algorithm(graph, shopping_list, product_prices, item_to_stores, population_size=50, num_generations=100):
    population = initialize_population(shopping_list, item_to_stores, population_size)
    
    for generation in range(num_generations):
        fitness_scores = [calculate_cost(ind, graph, product_prices, shopping_list) for ind in population]
        valid_scores = [(score, i) for i, (score, _, _) in enumerate(fitness_scores) if score != float('inf')]
        if valid_scores:
            best_score, best_idx = min(valid_scores)
            print(f"Generation {generation}: Best Cost = {best_score:.2f}")
        
        parents = [population[i] for _, i in sorted(valid_scores)[:population_size // 2]]
        next_population = parents.copy()
        
        while len(next_population) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            child = mutation(child, item_to_stores)
            next_population.append(child)
        
        population = next_population
    
    final_scores = [calculate_cost(ind, graph, product_prices, shopping_list) for ind in population]
    valid_scores = [(score, i) for i, (score, _, _) in enumerate(final_scores) if score != float('inf')]
    if not valid_scores:
        raise ValueError("Hiçbir çözüm tüm ürünleri kapsayamadı!")
    best_score, best_idx = min(valid_scores)
    optimal_solution = population[best_idx]
    total_cost = final_scores[best_idx][0]
    total_price = sum(price for _, price in final_scores[best_idx][1].values())
    item_sources = final_scores[best_idx][1]
    optimal_route = final_scores[best_idx][2]
    
    return optimal_solution, total_cost, total_price, item_sources, optimal_route

### Algortimayı Çalıştır

In [42]:

optimal_solution, total_cost, total_price, item_sources, optimal_route = genetic_algorithm(
    G_ga, shopping_list, product_prices, item_to_stores, population_size=50, num_generations=100
)

# Sonuçları Yazdır
from IPython.display import display
display(optimal_route)

print("+++++++++++++++++++++++")

print("\n=== Genetik Algortima Sonuçları ===")
print("Optimal Rota:")
for i, store in enumerate(optimal_route, 1):
    print(f"  {i}. {store}")
print(f"\nToplam Maliyet (Ulaşım + Ürün): {total_cost:.2f} TL")
print(f"Toplam Ürün Fiyatı: {total_price:.2f} TL")
print("\nÜrün Kaynakları:")
for item in shopping_list:
    if item in item_sources:
        store, price = item_sources[item]
        print(f"  {item}: {store} (Fiyat: {price:.2f} TL)")
    else:
        print(f"  {item}: Bulunamadı")
missing_items = set(shopping_list) - set(item_sources.keys())
if not missing_items:
    print("\nTüm ürünler başarıyla alındı!")
else:
    print("\nEksik ürünler:", missing_items)

# Görselleştirme
node_x = [pos_ga[node][0] for node in G_ga.nodes()]
node_y = [pos_ga[node][1] for node in G_ga.nodes()]
node_color = ["black" if node in optimal_route else kategori_renkleri.get(G_ga.nodes[node]['kategori_ga'], 'gray') for node in G_ga.nodes()]
node_text = [node if node in optimal_route else "" for node in G_ga.nodes()]

edge_x = []
edge_y = []
for i in range(len(optimal_route) - 1):
    x0, y0 = pos_ga[optimal_route[i]]
    x1, y1 = pos_ga[optimal_route[i + 1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig = go.Figure()
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, line=dict(width=2, color='gray'), mode='lines', name="Rota"))
fig.add_trace(go.Scatter(
    x=node_x, y=node_y, mode='markers+text',
    marker=dict(size=10, color=node_color),
    text=node_text, textposition="top center", hoverinfo="text"
))
fig.update_layout(
    title="Optimal Alışveriş Rotası ve Ürün Seçimi (Genetik Algoritma)",
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)
fig.write_html("genetic_algorithm.html")
webbrowser.open("genetic_algorithm.html")

Generation 0: Best Cost = 6710.38
Generation 1: Best Cost = 6022.83
Generation 2: Best Cost = 3418.68
Generation 3: Best Cost = 3114.33
Generation 4: Best Cost = 3114.33
Generation 5: Best Cost = 3017.34
Generation 6: Best Cost = 2390.07
Generation 7: Best Cost = 1928.26


Generation 8: Best Cost = 1928.26
Generation 9: Best Cost = 1928.26
Generation 10: Best Cost = 1928.26
Generation 11: Best Cost = 1928.26
Generation 12: Best Cost = 1829.95
Generation 13: Best Cost = 1829.95
Generation 14: Best Cost = 1814.14
Generation 15: Best Cost = 1741.79
Generation 16: Best Cost = 1694.05
Generation 17: Best Cost = 1603.21
Generation 18: Best Cost = 1603.21
Generation 19: Best Cost = 1603.21
Generation 20: Best Cost = 1603.21
Generation 21: Best Cost = 1603.21
Generation 22: Best Cost = 1603.21
Generation 23: Best Cost = 1571.08
Generation 24: Best Cost = 1538.03
Generation 25: Best Cost = 1538.03
Generation 26: Best Cost = 1538.03
Generation 27: Best Cost = 1538.03
Generation 28: Best Cost = 1538.03
Generation 29: Best Cost = 1520.28
Generation 30: Best Cost = 1479.36
Generation 31: Best Cost = 1479.36
Generation 32: Best Cost = 1479.36
Generation 33: Best Cost = 1479.36
Generation 34: Best Cost = 1479.36
Generation 35: Best Cost = 1479.36
Generation 36: Best Co

['Elektronik_Magaza_13',
 'Gida_Magaza_13',
 'Ev_Magaza_0',
 'Giysi_Magaza_22',
 'Kozmetik_Magaza_32',
 'Ev_Magaza_27',
 'Gida_Magaza_37',
 'Giysi_Magaza_28']

+++++++++++++++++++++++

=== Genetik Algortima Sonuçları ===
Optimal Rota:
  1. Elektronik_Magaza_13
  2. Gida_Magaza_13
  3. Ev_Magaza_0
  4. Giysi_Magaza_22
  5. Kozmetik_Magaza_32
  6. Ev_Magaza_27
  7. Gida_Magaza_37
  8. Giysi_Magaza_28

Toplam Maliyet (Ulaşım + Ürün): 1431.42 TL
Toplam Ürün Fiyatı: 1024.88 TL

Ürün Kaynakları:
  Bant: Ev_Magaza_27 (Fiyat: 121.04 TL)
  Sabun: Ev_Magaza_0 (Fiyat: 156.79 TL)
  Süt: Gida_Magaza_37 (Fiyat: 62.63 TL)
  Peynir: Gida_Magaza_13 (Fiyat: 30.40 TL)
  Hırka: Giysi_Magaza_28 (Fiyat: 172.88 TL)
  Elbise: Giysi_Magaza_22 (Fiyat: 65.51 TL)
  Bot: Giysi_Magaza_22 (Fiyat: 92.61 TL)
  Tablet: Elektronik_Magaza_13 (Fiyat: 268.61 TL)
  Oje: Kozmetik_Magaza_32 (Fiyat: 54.41 TL)

Tüm ürünler başarıyla alındı!


True